In [75]:
import pandas as pd
import datetime
import pyodbc
import pyodbc as db  #SQL Server

In [76]:
Team = pd.read_csv("E:\SEM-3\CS 779\Project\IPL\Team.csv")
#Removed unwanted columns
Team = Team.drop(['Team_SK'],axis=1)
#Remove all spaced to add data easily in SQL server
for i,row in Team.iterrows():
    Team.at[i,"Team_Name"] = row.Team_Name.replace(" ","_")

In [77]:
Match = pd.read_csv("E:\SEM-3\CS 779\Project\IPL\Match.csv",encoding='latin1')

In [78]:
Match = Match[['match_id','match_date','Season_Year','Team1','Team2','Toss_Winner','Toss_Name','match_winner','ManOfMach','Venue_Name','City_Name','Country_Name']]

In [79]:
Match['match_winner'] = Match["match_winner"].fillna('Cancelled')
Match['ManOfMach'] = Match["ManOfMach"].fillna('Match_Was_Cancelled')
Match['Toss_Winner'] = Match["Toss_Winner"].fillna('Match_Was_Cancelled')
Match['Venue_Name'] = Match["Venue_Name"].fillna('Match_Was_Cancelled')

In [80]:
for i,row in Match.iterrows():
    Match.at[i,"Team1"] = row.Team1.replace(" ","_")
    Match.at[i,"Team2"] = row.Team2.replace(" ","_")
    Match.at[i,"Toss_Winner"] = row.Toss_Winner.replace(" ","_")
    Match.at[i,"match_winner"] = row.match_winner.replace(" ","_")
    Match.at[i,"ManOfMach"] = row.ManOfMach.replace(" ","_")
    Match.at[i,"Venue_Name"] = row.Venue_Name.replace(" ","_")
    Match.at[i,"City_Name"] = row.City_Name.replace(" ","_")

In [81]:
for i,row in Match.iterrows():
    Match.at[i,"ManOfMach"] = row.ManOfMach.replace("-","_")

In [82]:
Match['Team1'] = Match['Team1'].map(Team.set_index('Team_Name')['Team_Id'])
Match['Team2'] = Match['Team2'].map(Team.set_index('Team_Name')['Team_Id'])
Match['Toss_Winner'] = Match['Toss_Winner'].map(Team.set_index('Team_Name')['Team_Id'])
Match['match_winner'] = Match['match_winner'].map(Team.set_index('Team_Name')['Team_Id'])


In [83]:
Match['match_winner'] = Match["match_winner"].fillna(0)
Match['ManOfMach'] = Match["ManOfMach"].fillna(0)
Match['Toss_Winner'] = Match["Toss_Winner"].fillna(0)
Match['Toss_Name'] = Match["Toss_Name"].fillna('No_Results')

In [84]:
Match[['Toss_Winner']] = Match[['Toss_Winner']].astype(int)
Match[['match_winner']] = Match[['match_winner']].astype(int)


In [85]:
Match

,match_id,match_date,Season_Year,Team1,Team2,Toss_Winner,Toss_Name,match_winner,ManOfMach,Venue_Name,City_Name,Country_Name
0,335987,4/18/2008,2008,2,1,2,field,1,BB_McCullum,M_Chinnaswamy_Stadium,Bangalore,India
1,335988,4/19/2008,2008,4,3,3,bat,3,MEK_Hussey,"Punjab_Cricket_Association_Stadium,_Mohali",Chandigarh,India
2,335989,4/19/2008,2008,6,5,5,bat,6,MF_Maharoof,Feroz_Shah_Kotla,Delhi,India
3,335990,4/20/2008,2008,7,2,7,bat,2,MV_Boucher,Wankhede_Stadium,Mumbai,India
4,335991,4/20/2008,2008,1,8,8,bat,1,DJ_Hussey,Eden_Gardens,Kolkata,India
...,...,...,...,...,...,...,...,...,...,...,...,...
632,1082646,5/14/2017,2017,6,2,2,Bat,2,HV_Patel,Feroz_Shah_Kotla,Delhi,India
633,1082647,5/16/2017,2017,7,12,7,Field,12,Washington_Sundar,Wankhede_Stadium,Mumbai,India
634,1082648,5/17/2017,2017,11,1,1,Field,1,NM_Coulter_Nile,M_Chinnaswamy_Stadium,Bengaluru,India
635,1082649,5/19/2017,2017,7,1,7,Field,7,KV_Sharma,M_Chinnaswamy_Stadium,Bengaluru,India


In [86]:
ball = pd.read_csv("E:\SEM-3\CS 779\Project\IPL\Ball_By_Ball.csv",encoding='latin1')
weather = pd.read_csv("E:\SEM-3\CS 779\Project\weather\DailyDelhiClimateTrain.csv",encoding='latin1')
Ball_date = ball[['MatchDateSK']]
Weather_date = weather[['date']]
Weather_date = Weather_date.drop([1461])
Weather_date['Date'] = ''
for i,row in Weather_date.iterrows():
        date = Weather_date.at[i,"date"] 
        Weather_date.at[i,"Date"]  = datetime.datetime.strptime(date, '%m/%d/%Y').strftime('%Y%m%d')

In [87]:
Weather_date = Weather_date[['Date']]

In [88]:
Ball_date = Ball_date.rename(columns={'MatchDateSK':'Date'})

In [89]:
Date = Ball_date.append(Weather_date, ignore_index = True) 

In [90]:
Date

,Date
0,20130420
1,20130420
2,20130420
3,20130420
4,20130420
...,...
151907,20161227
151908,20161228
151909,20161229
151910,20161230


In [91]:
Date['Year'] = Date['Date'].astype(str).str[0:4]
Date['Month'] = Date['Date'].astype(str).str[4:6]
Date['Day'] = Date['Date'].astype(str).str[6:8]

In [92]:
Date= Date.drop_duplicates()

In [93]:
Date

,Date,Year,Month,Day
0,20130420,2013,04,20
31,20130512,2013,05,12
114,20130422,2013,04,22
117,20150412,2015,04,12
219,20150503,2015,05,03
...,...,...,...,...
151907,20161227,2016,12,27
151908,20161228,2016,12,28
151909,20161229,2016,12,29
151910,20161230,2016,12,30


In [94]:
Date['Date_Id'] = [i for i in range(1,len(Date)+1)]

C:\Users\aksha\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [95]:
Date['Full_date'] = Date.apply(lambda row: row.Year +'-'+row.Month+'-'+row.Day,axis= 1)

C:\Users\aksha\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [96]:
Date = Date[['Date_Id','Full_date','Year','Month','Day','Date']]

In [97]:
Date


,Date_Id,Full_date,Year,Month,Day,Date
0,1,2013-04-20,2013,04,20,20130420
31,2,2013-05-12,2013,05,12,20130512
114,3,2013-04-22,2013,04,22,20130422
117,4,2015-04-12,2015,04,12,20150412
219,5,2015-05-03,2015,05,03,20150503
...,...,...,...,...,...,...
151907,1907,2016-12-27,2016,12,27,20161227
151908,1908,2016-12-28,2016,12,28,20161228
151909,1909,2016-12-29,2016,12,29,20161229
151910,1910,2016-12-30,2016,12,30,20161230


In [98]:
weather

,date,meantemp,humidity,wind_speed,meanpressure
0,1/1/2013,10.000000,84.500000,0.000000,1015.666667
1,1/2/2013,7.400000,92.000000,2.980000,1017.800000
2,1/3/2013,7.166667,87.000000,4.633333,1018.666667
3,1/4/2013,8.666667,71.333333,1.233333,1017.166667
4,1/5/2013,6.000000,86.833333,3.700000,1016.500000
...,...,...,...,...,...
1457,12/28/2016,17.217391,68.043478,3.547826,1015.565217
1458,12/29/2016,15.238095,87.857143,6.000000,1016.904762
1459,12/30/2016,14.095238,89.666667,6.266667,1017.904762
1460,12/31/2016,15.052632,87.000000,7.325000,1016.100000


In [99]:
weather = weather.drop([1461])

In [100]:
for i,row in weather.iterrows():
        date = weather.at[i,"date"] 
        weather.at[i,"Date_desc"]  = datetime.datetime.strptime(date, '%m/%d/%Y').strftime('%Y%m%d')

In [101]:
weather

,date,meantemp,humidity,wind_speed,meanpressure,Date_desc
0,1/1/2013,10.000000,84.500000,0.000000,1015.666667,20130101
1,1/2/2013,7.400000,92.000000,2.980000,1017.800000,20130102
2,1/3/2013,7.166667,87.000000,4.633333,1018.666667,20130103
3,1/4/2013,8.666667,71.333333,1.233333,1017.166667,20130104
4,1/5/2013,6.000000,86.833333,3.700000,1016.500000,20130105
...,...,...,...,...,...,...
1456,12/27/2016,16.850000,67.550000,8.335000,1017.200000,20161227
1457,12/28/2016,17.217391,68.043478,3.547826,1015.565217,20161228
1458,12/29/2016,15.238095,87.857143,6.000000,1016.904762,20161229
1459,12/30/2016,14.095238,89.666667,6.266667,1017.904762,20161230


In [102]:
Match_new = Match[Match.City_Name == 'Delhi']

In [103]:
for i,row in Match_new.iterrows():
        date = Match_new.at[i,"match_date"] 
        Match_new.at[i,"Date_desc"]  = datetime.datetime.strptime(date, '%m/%d/%Y').strftime('%Y%m%d')

In [104]:
Match_new

,match_id,match_date,Season_Year,Team1,Team2,Toss_Winner,Toss_Name,match_winner,ManOfMach,Venue_Name,City_Name,Country_Name,Date_desc
2,335989,4/19/2008,2008,6,5,5,bat,6,MF_Maharoof,Feroz_Shah_Kotla,Delhi,India,20080419
16,336003,4/30/2008,2008,6,2,2,field,6,GD_McGrath,Feroz_Shah_Kotla,Delhi,India,20080430
27,336014,5/8/2008,2008,6,3,3,field,3,MS_Dhoni,Feroz_Shah_Kotla,Delhi,India,20080508
38,336025,5/15/2008,2008,6,8,8,field,6,A_Mishra,Feroz_Shah_Kotla,Delhi,India,20080515
40,336027,5/17/2008,2008,6,4,6,bat,4,DPMD_Jayawardene,Feroz_Shah_Kotla,Delhi,India,20080517
49,336037,5/24/2008,2008,6,7,6,field,6,KD_Karthik,Feroz_Shah_Kotla,Delhi,India,20080524
123,419119,3/17/2010,2010,6,7,6,field,7,SR_Tendulkar,Feroz_Shah_Kotla,Delhi,India,20100317
125,419121,3/19/2010,2010,6,3,6,bat,3,ML_Hayden,Feroz_Shah_Kotla,Delhi,India,20100319
140,419136,3/29/2010,2010,6,1,6,bat,6,DA_Warner,Feroz_Shah_Kotla,Delhi,India,20100329
143,419139,3/31/2010,2010,6,5,6,bat,6,KD_Karthik,Feroz_Shah_Kotla,Delhi,India,20100331


In [105]:
weather_final = Match_new.merge(weather,on='Date_desc',how = 'inner')


In [106]:
weather_final

,match_id,match_date,Season_Year,Team1,Team2,Toss_Winner,Toss_Name,match_winner,ManOfMach,Venue_Name,City_Name,Country_Name,Date_desc,date,meantemp,humidity,wind_speed,meanpressure
0,598006,4/6/2013,2013,6,5,5,bat,5,R_Dravid,Feroz_Shah_Kotla,Delhi,India,20130406,4/6/2013,25.857143,29.857143,11.900000,1006.142857
1,598015,4/12/2013,2013,6,11,6,bat,11,A_Mishra,Feroz_Shah_Kotla,Delhi,India,20130412,4/12/2013,28.857143,32.571429,6.342857,1007.571429
2,598025,4/18/2013,2013,6,3,3,bat,3,MEK_Hussey,Feroz_Shah_Kotla,Delhi,India,20130418,4/18/2013,30.285714,30.285714,4.757143,1002.857143
3,598029,4/21/2013,2013,6,7,7,bat,6,V_Sehwag,Feroz_Shah_Kotla,Delhi,India,20130421,4/21/2013,27.666667,38.666667,13.883333,1006.833333
4,598059,5/10/2013,2013,6,2,6,field,2,JD_Unadkat,Feroz_Shah_Kotla,Delhi,India,20130510,5/10/2013,34.000000,27.714286,9.528571,1000.142857
5,598064,4/23/2013,2013,6,4,4,field,4,Harmeet_Singh,Feroz_Shah_Kotla,Delhi,India,20130423,4/23/2013,28.625000,44.125000,4.625000,1009.875000
6,598075,5/21/2013,2013,3,7,3,bat,3,MEK_Hussey,Feroz_Shah_Kotla,Delhi,India,20130521,5/21/2013,36.000000,19.000000,11.371429,998.666667
7,598076,5/22/2013,2013,5,11,11,bat,5,BJ_Hodge,Feroz_Shah_Kotla,Delhi,India,20130522,5/22/2013,36.750000,22.125000,17.587500,998.625000
8,733980,5/3/2014,2014,6,5,5,field,5,KK_Nair,Feroz_Shah_Kotla,Delhi,India,20140503,5/3/2014,31.750000,33.375000,6.475000,1003.625000
9,733986,5/5/2014,2014,6,3,3,field,3,DR_Smith,Feroz_Shah_Kotla,Delhi,India,20140505,5/5/2014,29.750000,53.250000,9.037500,1005.750000


In [107]:
weather_final['Team_batting_1']=''
weather_final['run_scored_by_team_1']=''
weather_final['extra_scored_by_team_1']=''
weather_final['Total_team_1']=''
weather_final['Team_batting_2']=''
weather_final['run_scored_by_team_2']=''
weather_final['extra_scored_by_team_2']=''
weather_final['Total_team_2']=''

In [108]:
Ball_by_Ball = pd.read_csv("E:\SEM-3\CS 779\Project\IPL\Ball_By_Ball.csv",encoding='latin1')

In [109]:
for i,row in Ball_by_Ball.iterrows():
    if Ball_by_Ball.at[i,"Team_Batting"] == 'Kolkata Knight Riders':
        Ball_by_Ball.at[i,"Team_Batting"] = 1
    elif Ball_by_Ball.at[i,"Team_Batting"] == 'Royal Challengers Bangalore':
        Ball_by_Ball.at[i,"Team_Batting"] = 2
    elif Ball_by_Ball.at[i,"Team_Batting"] == 'Chennai Super Kings':
        Ball_by_Ball.at[i,"Team_Batting"] = 3
    elif Ball_by_Ball.at[i,"Team_Batting"] == 'Kings XI Punjab':
        Ball_by_Ball.at[i,"Team_Batting"] = 4
    elif Ball_by_Ball.at[i,"Team_Batting"] == 'Rajasthan Royals':
        Ball_by_Ball.at[i,"Team_Batting"] = 5
    elif Ball_by_Ball.at[i,"Team_Batting"] == 'Delhi Daredevils':
        Ball_by_Ball.at[i,"Team_Batting"] = 6     
    elif Ball_by_Ball.at[i,"Team_Batting"] == 'Mumbai Indians':
        Ball_by_Ball.at[i,"Team_Batting"] = 7
    elif Ball_by_Ball.at[i,"Team_Batting"] == 'Deccan Chargers':
        Ball_by_Ball.at[i,"Team_Batting"] = 8
    elif Ball_by_Ball.at[i,"Team_Batting"] == 'Kochi Tuskers Kerala':
        Ball_by_Ball.at[i,"Team_Batting"] = 9
    elif Ball_by_Ball.at[i,"Team_Batting"] == 'Pune Warriors':
        Ball_by_Ball.at[i,"Team_Batting"] = 10
    elif Ball_by_Ball.at[i,"Team_Batting"] == 'Sunrisers Hyderabad':
        Ball_by_Ball.at[i,"Team_Batting"] = 11
    elif Ball_by_Ball.at[i,"Team_Batting"] == 'Rising Pune Supergiants':
        Ball_by_Ball.at[i,"Team_Batting"] = 12
    elif Ball_by_Ball.at[i,"Team_Batting"] == 'Gujarat Lions':
        Ball_by_Ball.at[i,"Team_Batting"] = 13

In [110]:
for i,row in Ball_by_Ball.iterrows():
    if Ball_by_Ball.at[i,"Team_Bowling"] == 'Kolkata Knight Riders':
        Ball_by_Ball.at[i,"Team_Bowling"] = 1
    elif Ball_by_Ball.at[i,"Team_Bowling"] == 'Royal Challengers Bangalore':
        Ball_by_Ball.at[i,"Team_Bowling"] = 2
    elif Ball_by_Ball.at[i,"Team_Bowling"] == 'Chennai Super Kings':
        Ball_by_Ball.at[i,"Team_Bowling"] = 3
    elif Ball_by_Ball.at[i,"Team_Bowling"] == 'Kings XI Punjab':
        Ball_by_Ball.at[i,"Team_Bowling"] = 4
    elif Ball_by_Ball.at[i,"Team_Bowling"] == 'Rajasthan Royals':
        Ball_by_Ball.at[i,"Team_Bowling"] = 5
    elif Ball_by_Ball.at[i,"Team_Bowling"] == 'Delhi Daredevils':
        Ball_by_Ball.at[i,"Team_Bowling"] = 6     
    elif Ball_by_Ball.at[i,"Team_Bowling"] == 'Mumbai Indians':
        Ball_by_Ball.at[i,"Team_Bowling"] = 7
    elif Ball_by_Ball.at[i,"Team_Bowling"] == 'Deccan Chargers':
        Ball_by_Ball.at[i,"Team_Bowling"] = 8
    elif Ball_by_Ball.at[i,"Team_Bowling"] == 'Kochi Tuskers Kerala':
        Ball_by_Ball.at[i,"Team_Bowling"] = 9
    elif Ball_by_Ball.at[i,"Team_Bowling"] == 'Pune Warriors':
        Ball_by_Ball.at[i,"Team_Bowling"] = 10
    elif Ball_by_Ball.at[i,"Team_Bowling"] == 'Sunrisers Hyderabad':
        Ball_by_Ball.at[i,"Team_Bowling"] = 11
    elif Ball_by_Ball.at[i,"Team_Bowling"] == 'Rising Pune Supergiants':
        Ball_by_Ball.at[i,"Team_Bowling"] = 12
    elif Ball_by_Ball.at[i,"Team_Bowling"] == 'Gujarat Lions':
        Ball_by_Ball.at[i,"Team_Bowling"] = 13

In [111]:
Ball_by_Ball = Ball_by_Ball[['MatcH_id','Innings_No','Team_Batting','Team_Bowling','Runs_Scored','Extra_runs']]

In [112]:
Ball_by_Ball

,MatcH_id,Innings_No,Team_Batting,Team_Bowling,Runs_Scored,Extra_runs
0,598028,1,5,2,4,0
1,598028,1,5,2,1,0
2,598028,1,5,2,1,0
3,598028,1,5,2,1,0
4,598028,1,5,2,0,0
...,...,...,...,...,...,...
150446,598028,1,5,2,1,0
150447,598028,1,5,2,2,0
150448,598028,1,5,2,4,0
150449,598028,1,5,2,0,0


In [113]:
weather_final

,match_id,match_date,Season_Year,Team1,Team2,Toss_Winner,Toss_Name,match_winner,ManOfMach,Venue_Name,...,wind_speed,meanpressure,Team_batting_1,run_scored_by_team_1,extra_scored_by_team_1,Total_team_1,Team_batting_2,run_scored_by_team_2,extra_scored_by_team_2,Total_team_2
0,598006,4/6/2013,2013,6,5,5,bat,5,R_Dravid,Feroz_Shah_Kotla,...,11.900000,1006.142857,,,,,,,,
1,598015,4/12/2013,2013,6,11,6,bat,11,A_Mishra,Feroz_Shah_Kotla,...,6.342857,1007.571429,,,,,,,,
2,598025,4/18/2013,2013,6,3,3,bat,3,MEK_Hussey,Feroz_Shah_Kotla,...,4.757143,1002.857143,,,,,,,,
3,598029,4/21/2013,2013,6,7,7,bat,6,V_Sehwag,Feroz_Shah_Kotla,...,13.883333,1006.833333,,,,,,,,
4,598059,5/10/2013,2013,6,2,6,field,2,JD_Unadkat,Feroz_Shah_Kotla,...,9.528571,1000.142857,,,,,,,,
5,598064,4/23/2013,2013,6,4,4,field,4,Harmeet_Singh,Feroz_Shah_Kotla,...,4.625000,1009.875000,,,,,,,,
6,598075,5/21/2013,2013,3,7,3,bat,3,MEK_Hussey,Feroz_Shah_Kotla,...,11.371429,998.666667,,,,,,,,
7,598076,5/22/2013,2013,5,11,11,bat,5,BJ_Hodge,Feroz_Shah_Kotla,...,17.587500,998.625000,,,,,,,,
8,733980,5/3/2014,2014,6,5,5,field,5,KK_Nair,Feroz_Shah_Kotla,...,6.475000,1003.625000,,,,,,,,
9,733986,5/5/2014,2014,6,3,3,field,3,DR_Smith,Feroz_Shah_Kotla,...,9.037500,1005.750000,,,,,,,,


In [114]:
Team_batting_1 = []
for i,row in weather_final.iterrows():
        a = Ball_by_Ball.loc[(Ball_by_Ball['MatcH_id'] == weather_final.at[i,"match_id"]) & (Ball_by_Ball['Innings_No'] == 1),'Team_Batting'].iloc[0]
        Team_batting_1.append(a)
Team_batting_2 = []
for i,row in weather_final.iterrows():
        a = Ball_by_Ball.loc[(Ball_by_Ball['MatcH_id'] == weather_final.at[i,"match_id"]) & (Ball_by_Ball['Innings_No'] == 2),'Team_Batting'].iloc[0]
        Team_batting_2.append(a)
run_scored_by_team_1 = []
for i,row in weather_final.iterrows():
        a = Ball_by_Ball.loc[(Ball_by_Ball['MatcH_id'] == weather_final.at[i,"match_id"]) & (Ball_by_Ball['Innings_No'] == 1),'Runs_Scored'].sum()
        run_scored_by_team_1.append(a)
run_scored_by_team_2 = []
for i,row in weather_final.iterrows():
        a = Ball_by_Ball.loc[(Ball_by_Ball['MatcH_id'] == weather_final.at[i,"match_id"]) & (Ball_by_Ball['Innings_No'] == 2),'Runs_Scored'].sum()
        run_scored_by_team_2.append(a)
extra_scored_by_team_1 = []
for i,row in weather_final.iterrows():
        a = Ball_by_Ball.loc[(Ball_by_Ball['MatcH_id'] == weather_final.at[i,"match_id"]) & (Ball_by_Ball['Innings_No'] == 1),'Extra_runs'].sum()
        extra_scored_by_team_1.append(a)
extra_scored_by_team_2 = []
for i,row in weather_final.iterrows():
        a = Ball_by_Ball.loc[(Ball_by_Ball['MatcH_id'] == weather_final.at[i,"match_id"]) & (Ball_by_Ball['Innings_No'] == 2),'Extra_runs'].sum()
        extra_scored_by_team_2.append(a)

In [67]:
print(Team_batting_1)
print(Team_batting_2)
print(run_scored_by_team_1)
print(run_scored_by_team_2)
print(extra_scored_by_team_1)
print(extra_scored_by_team_2)


['5', '6', '3', '7', '2', '6', '3', '11', '6', '6', '6', '6', '6', '6', '6', '6', '6', '4', '4', '6', '13', '6', '6', '11', '13']
['6', '11', '6', '6', '6', '4', '7', '5', '5', '3', '1', '11', '4', '5', '1', '7', '2', '6', '6', '7', '6', '1', '12', '1', '11']
[156, 107, 160, 154, 178, 113, 191, 126, 148, 167, 152, 138, 159, 175, 133, 182, 87, 110, 107, 152, 169, 176, 154, 156, 157]
[149, 112, 79, 161, 168, 115, 139, 131, 152, 170, 149, 33, 150, 180, 140, 142, 97, 111, 103, 151, 168, 146, 155, 131, 161]
[9, 7, 9, 3, 5, 7, 1, 6, 4, 11, 8, 5, 5, 9, 13, 8, 8, 8, 4, 12, 3, 10, 8, 6, 5]
[11, 3, 4, 4, 11, 6, 5, 4, 4, 11, 12, 11, 15, 6, 7, 11, 2, 8, 10, 3, 3, 13, 11, 9, 2]


In [115]:
weather_final['Team_batting_1']=Team_batting_1
weather_final['run_scored_by_team_1']=run_scored_by_team_1
weather_final['extra_scored_by_team_1']=extra_scored_by_team_1
weather_final['Total_team_1']=[x + y for x, y in zip((run_scored_by_team_1), (extra_scored_by_team_1))] 
weather_final['Team_batting_2']=Team_batting_2
weather_final['run_scored_by_team_2']=run_scored_by_team_2
weather_final['extra_scored_by_team_2']=extra_scored_by_team_2
weather_final['Total_team_2']=[x + y for x, y in zip((run_scored_by_team_2), (extra_scored_by_team_2))] 

In [116]:
weather_final[:10]

,match_id,match_date,Season_Year,Team1,Team2,Toss_Winner,Toss_Name,match_winner,ManOfMach,Venue_Name,...,wind_speed,meanpressure,Team_batting_1,run_scored_by_team_1,extra_scored_by_team_1,Total_team_1,Team_batting_2,run_scored_by_team_2,extra_scored_by_team_2,Total_team_2
0,598006,4/6/2013,2013,6,5,5,bat,5,R_Dravid,Feroz_Shah_Kotla,...,11.900000,1006.142857,5,156,9,165,6,149,11,160
1,598015,4/12/2013,2013,6,11,6,bat,11,A_Mishra,Feroz_Shah_Kotla,...,6.342857,1007.571429,6,107,7,114,11,112,3,115
2,598025,4/18/2013,2013,6,3,3,bat,3,MEK_Hussey,Feroz_Shah_Kotla,...,4.757143,1002.857143,3,160,9,169,6,79,4,83
3,598029,4/21/2013,2013,6,7,7,bat,6,V_Sehwag,Feroz_Shah_Kotla,...,13.883333,1006.833333,7,154,3,157,6,161,4,165
4,598059,5/10/2013,2013,6,2,6,field,2,JD_Unadkat,Feroz_Shah_Kotla,...,9.528571,1000.142857,2,178,5,183,6,168,11,179
5,598064,4/23/2013,2013,6,4,4,field,4,Harmeet_Singh,Feroz_Shah_Kotla,...,4.625000,1009.875000,6,113,7,120,4,115,6,121
6,598075,5/21/2013,2013,3,7,3,bat,3,MEK_Hussey,Feroz_Shah_Kotla,...,11.371429,998.666667,3,191,1,192,7,139,5,144
7,598076,5/22/2013,2013,5,11,11,bat,5,BJ_Hodge,Feroz_Shah_Kotla,...,17.587500,998.625000,11,126,6,132,5,131,4,135
8,733980,5/3/2014,2014,6,5,5,field,5,KK_Nair,Feroz_Shah_Kotla,...,6.475000,1003.625000,6,148,4,152,5,152,4,156
9,733986,5/5/2014,2014,6,3,3,field,3,DR_Smith,Feroz_Shah_Kotla,...,9.037500,1005.750000,6,167,11,178,3,170,11,181


In [70]:
weather_final.to_csv(r'E:\SEM-3\CS 779\Project\Weather_effect.csv')

25